# SMS Spam Detection

In [3]:
import pandas as pd

# Read in data from csv
data = pd.read_csv("datasets/spam.csv", encoding='latin-1')

# Drop columns with NaN values
for col in data.columns:
    if 'Unnamed' in col:
        data.drop(columns=col, inplace=True)

# Rename columns to useful names
data = data.rename(columns={"v1":"Label", "v2":"Text"})

# Add an encoding for the labels: ham = 0, spam = 1
enc_labels = []
for item in data["Label"]:
    enc_labels.append(0 if item == "ham" else 1)
data.insert(loc=2, column="encoded_label", value=enc_labels)
print(data.head())


  Label                                               Text  encoded_label
0   ham  Go until jurong point, crazy.. Available only ...              0
1   ham                      Ok lar... Joking wif u oni...              0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...              1
3   ham  U dun say so early hor... U c already then say...              0
4   ham  Nah I don't think he goes to usf, he lives aro...              0


In [4]:
# Find the average number of tokens in all sentences
sentences = data["Text"].values

avg_sentence_len = 0
for sentence in sentences:
    avg_sentence_len += len(sentence.split()) # adding the number of words in each sentence
avg_sentence_len = avg_sentence_len // len(sentences) # divide by the total number of sentences

In [5]:
# Find the total number of unique words
s = set()
for sentence in sentences:
    for word in sentence.split():
        s.add(word)
num_unique_words = len(s)
print(num_unique_words)

15585


In [6]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data["Text"], data["encoded_label"], train_size=0.8, random_state=42)

In [73]:
import tensorflow as tf

text_vec = tf.keras.layers.TextVectorization(max_tokens=num_unique_words, standardize="lower_and_strip_punctuation", 
                                       output_sequence_length=avg_sentence_len, output_mode="int")
text_vec.adapt(X_train)
embed_layer = tf.keras.layers.Embedding(input_dim=num_unique_words, output_dim=128, input_length=avg_sentence_len,
                                        embeddings_initializer="uniform")

input_layer = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
vec_layer = text_vec(input_layer)
embedding_layer = embed_layer(vec_layer)
# x = tf.keras.layers.Dropout(0.75)(embedding_layer)
x = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
x = tf.keras.layers.Dropout(0.5)(x)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=5)

AttributeError: module 'keras.api._v2.keras.activations' has no attribute 'relu6'

In [69]:
y_pred = model.predict(X_test)
y_pred = y_pred.astype(int)

y_pred1D = []
for i in range(len(y_pred)):
    y_pred1D.append(y_pred[i][0])

y_pred1D = pd.Series(y_pred1D)

print(y_pred1D.values)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print("Accuracy:", accuracy_score(y_pred=y_pred1D, y_true=y_test))
print("Precision:", precision_score(y_pred=y_pred1D, y_true=y_test))
print("Recall:", recall_score(y_pred=y_pred1D, y_true=y_test))
print("F1:", f1_score(y_pred=y_pred1D, y_true=y_test))


35/35 [==============================] - 1s 4ms/step
[0 0 0 ... 0 0 0]
Accuracy: 0.8654708520179372
Precision: 0.0
Recall: 0.0
F1: 0.0


C:\Users\Daniel Cruz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
